## Modelling

In this Notebook we can start modelling, with some data from our DB.

- To do this we can connect with our local DB using the `duckdb` library
- When a connection has been made we can start retrieving data from our DB.

### Setup

In [112]:
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm

: 

In [50]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/data_systems_project.duckdb")
%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [51]:
%sql SHOW ALL TABLES; # shows all available tables

,database,schema,name,column_names,column_types,temporary
0,data_systems_project,public,cbs_wijken,"[geometry, wijkcode, wijknaam, gemeentecode, g...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
1,data_systems_project,public,fire_stations_and_vehicles,"[Fire_Station, Vehicle, Vehicle_Type]","[VARCHAR, VARCHAR, VARCHAR]",False
2,data_systems_project,public,grond_data,"[geometry, id, locatie, amNummer, typeOnderzoe...","[VARCHAR, BIGINT, VARCHAR, VARCHAR, VARCHAR, V...",False
3,data_systems_project,public,service_areas,"[H_Verzorgingsgebied_ID, Verzorgingsgebied, LA...","[BIGINT, VARCHAR, DOUBLE, DOUBLE, VARCHAR]",False
4,data_systems_project,public,storm_deployments,"[Deployment_ID, Incident_ID, Vehicle_Type, Veh...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False
5,data_systems_project,public,storm_incidents,"[Incident_ID, Date, Incident_Starttime, Incide...","[BIGINT, TIMESTAMP_MS, TIME, TIME, TIME, DOUBL...",False


In [ ]:
## We can use SQL magic to retrieve data from our DB like so:
# %sql res << SELECT * FROM joined.deployment_incident_vehicles_weather
# res

In [53]:
# Or the more Pythonic way:

# Here we retrieve a table where KNMI weather data and Fire Department data is combined
storm_incidents = conn.execute(
    """
     SELECT * FROM public.storm_incidents """
).df()

# Close the database connection
conn.close()

In [55]:
incidents_weather_tree = pd.read_csv("../dsp-dagster/data/test.csv", low_memory=False)

In [58]:
incidents_weather_tree["Incident_Occurred"] = np.where(
    incidents_weather_tree["Incident_ID"].notna(), 1, 0
)
non_incident_data = incidents_weather_tree[
    incidents_weather_tree["Incident_Occurred"] == 0
]
incident_data = incidents_weather_tree[incidents_weather_tree["Incident_Occurred"] == 1]
incident_data_tree = incident_data[incident_data["Damage_Type"] == "Tree"]

# Filtering incident_data_tree for unique values of 'Incident_ID'
unique_incident_data_tree = incident_data_tree.drop_duplicates(subset=["Incident_ID"])

In [65]:
non_incident_data

,Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,...,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle,Incident_Occurred
0,240,2005-01-01,1,260,40.0,30,60,68,NaN,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,240,2005-01-01,2,230,30.0,30,60,65,NaN,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,240,2005-01-01,3,230,40.0,30,50,43,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,240,2005-01-01,4,220,40.0,40,50,38,NaN,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,240,2005-01-01,5,230,40.0,40,50,38,NaN,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171876,240,2024-01-11,20,360,30.0,30,60,49,NaN,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
171877,240,2024-01-11,21,10,30.0,30,60,48,NaN,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
171878,240,2024-01-11,22,20,30.0,30,50,46,NaN,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
171879,240,2024-01-11,23,20,20.0,30,50,43,NaN,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### Tree Data

In [63]:
tree_data = pd.read_parquet("../dsp-dagster/data/tree_data.parquet")
tree_data.head()

,geometry,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop
0,POINT (4.9046709047432087 52.3398135011992380),919933,03630980000301,Stadsdeel Zuid,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia)
1,POINT (4.9026920600233872 52.3400930246176159),919934,03630980000301,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus)
2,POINT (4.8552076308414502 52.3319844126534122),919935,03630980000311,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus)
3,POINT (4.9036703572006219 52.3488368070214634),919936,03630980000297,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,None,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus)
4,POINT (4.8758864240042872 52.3410557215088588),919937,03630980000306,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,None,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus)


#### Extract Longitude / Latitude

In [64]:
# Convert WKT geometries to Shapely Point objects
tree_data["geometry"] = tree_data["geometry"].apply(
    lambda x: loads(x) if x is not None else None
)

# Extract Lon and Lat from the Point objects
tree_data["LON"] = tree_data["geometry"].apply(
    lambda point: point.x if point is not None else None
)
tree_data["LAT"] = tree_data["geometry"].apply(
    lambda point: point.y if point is not None else None
)

# Drop the "geometry" column
tree_data = tree_data.drop("geometry", axis=1)
tree_data

,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,LON,LAT
0,919933,03630980000301,Stadsdeel Zuid,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),4.904671,52.339814
1,919934,03630980000301,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),4.902692,52.340093
2,919935,03630980000311,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),4.855208,52.331984
3,919936,03630980000297,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,None,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),4.903670,52.348837
4,919937,03630980000306,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,None,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),4.875886,52.341056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285884,4362405,03630980000272,R&E_VOR_Bomen,None,Gemeente Amsterdam,2023.0,Prunus serrulata 'Fugenzo',None,None,Boom vrij uitgroeiend,Bomen,Prunus,Kers (Prunus),4.890240,52.350689
285885,4365757,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.827666,52.381965
285886,4365758,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.828119,52.381788
285887,4365768,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.832787,52.382459


#### Manipulate tree data

In [39]:
tree_data = tree_data.drop(tree_data[tree_data["jaarVanAanleg"] == 0.0].index)
tree_data = tree_data.dropna(subset=["jaarVanAanleg"])
tree_data = pd.get_dummies(tree_data, columns=["boomhoogteklasseActueel"])

In [105]:
tree_data

,id,gbdBuurtId,typeBeheerderPlus,boomhoogteklasseActueel,typeEigenaarPlus,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,LON,LAT
0,919933,03630980000301,Stadsdeel Zuid,e. 15 tot 18 m.,Gemeente Amsterdam,1948.0,Tilia americana,None,None,Boom niet vrij uitgroeiend,Bomen,Tilia,Linde (Tilia),4.904671,52.339814
1,919934,03630980000301,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1978.0,Ulmus hollandica 'Vegeta',None,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),4.902692,52.340093
2,919935,03630980000311,Stadsdeel Zuid,c. 9 tot 12 m.,Gemeente Amsterdam,1990.0,Fraxinus excelsior 'Westhof's Glorie',"0,2 tot 0,3 m.",None,Boom niet vrij uitgroeiend,Bomen,Fraxinus,Es (Fraxinus),4.855208,52.331984
3,919936,03630980000297,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,2002.0,Ulmus glabra 'Lutescens',None,None,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),4.903670,52.348837
4,919937,03630980000306,Stadsdeel Zuid,b. 6 tot 9 m.,Gemeente Amsterdam,1985.0,Quercus robur,None,None,Boom niet vrij uitgroeiend,Bomen,Quercus,Eik (Quercus),4.875886,52.341056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285884,4362405,03630980000272,R&E_VOR_Bomen,None,Gemeente Amsterdam,2023.0,Prunus serrulata 'Fugenzo',None,None,Boom vrij uitgroeiend,Bomen,Prunus,Kers (Prunus),4.890240,52.350689
285885,4365757,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.827666,52.381965
285886,4365758,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.828119,52.381788
285887,4365768,03630980000193,R&E_VOR_Bomen,None,Gemeente Amsterdam,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.832787,52.382459


#### Sample random tree to non-incident data

In [ ]:
non_incident_data.loc[:, "LON"] = np.random.choice(
    tree_data["LON"], size=len(non_incident_data)
)
non_incident_data.loc[:, "LAT"] = np.random.choice(
    tree_data["LAT"], size=len(non_incident_data)
)

non_incident_sample = non_incident_data.sample(
    n=100 * len(unique_incident_data_tree), random_state=42
)

In [76]:
# Convert incident data and tree data to GeoDataFrames
incident_gdf = gpd.GeoDataFrame(
    unique_incident_data_tree,
    geometry=gpd.points_from_xy(
        unique_incident_data_tree.LON, unique_incident_data_tree.LAT
    ),
)
tree_gdf = gpd.GeoDataFrame(
    tree_data, geometry=gpd.points_from_xy(tree_data.LON, tree_data.LAT)
)

# Set the original CRS for both GeoDataFrames to WGS84 (EPSG:4326)
incident_gdf.set_crs(epsg=4326, inplace=True)
tree_gdf.set_crs(epsg=4326, inplace=True)

# Reproject to a suitable projected CRS (e.g., UTM zone 31N)
incident_gdf = incident_gdf.to_crs(epsg=32631)
tree_gdf = tree_gdf.to_crs(epsg=32631)

# Perform the spatial join
incident_with_trees = gpd.sjoin_nearest(incident_gdf, tree_gdf, distance_col="distance")

In [95]:
# incident_with_trees now contains incidents with the nearest tree's characteristics in the correct CRS

model_columns = [
    "Dd",
    "Fh",
    "Ff",
    "Fx",
    "T",
    "Td",
    "Sq",
    "Q",
    "Dr",
    "Rh",
    "P",
    "Vv",
    "N",
    "U",
    "Ix",
    "M",
    "R",
    "S",
    "O",
    "Y",
    "LON_left",
    "LAT_left",
    "boomhoogteklasseActueel",
    "jaarVanAanleg",
    "Incident_Occurred",
    "distance",
]
incident_with_trees_selected = incident_with_trees[model_columns]

incident_with_trees_selected["LON"] = incident_with_trees_selected["LON_left"]
incident_with_trees_selected["LAT"] = incident_with_trees_selected["LAT_left"]

# Dropping rows with any NaNs in the selected columns
incident_with_trees_cleaned = incident_with_trees_selected.dropna()
incident_with_trees_cleaned = incident_with_trees_cleaned.drop("LON_left", axis=1)
incident_with_trees_cleaned = incident_with_trees_cleaned.drop("LAT_left", axis=1)

/tmp/ipykernel_7450/3684192548.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_with_trees_selected["LON"] = incident_with_trees_selected["LON_left"]
/tmp/ipykernel_7450/3684192548.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_with_trees_selected["LAT"] = incident_with_trees_selected["LAT_left"]


In [96]:
incident_with_trees_cleaned

,Dd,Fh,Ff,Fx,T,Td,Sq,Q,Dr,Rh,...,R,S,O,Y,boomhoogteklasseActueel,jaarVanAanleg,Incident_Occurred,distance,LON,LAT
2317,240,160.0,160,210,99,13,8,195,0,0,...,0.0,0.0,0.0,0.0,a. tot 6 m.,2016.0,1,8.600111,4.826198,52.352551
10835,240,100.0,100,150,109,98,2,22,0,0,...,0.0,0.0,0.0,0.0,e. 15 tot 18 m.,1993.0,1,9.867678,4.930920,52.358682
11926,30,50.0,50,70,226,36,10,121,0,0,...,0.0,0.0,0.0,0.0,b. 6 tot 9 m.,1990.0,1,4.078773,4.885194,52.357166
12134,240,140.0,130,210,136,107,0,46,0,0,...,0.0,0.0,0.0,0.0,f. 18 tot 24 m.,1956.0,1,9.182905,4.871252,52.360666
13608,40,60.0,60,90,258,158,10,40,0,0,...,0.0,0.0,0.0,0.0,b. 6 tot 9 m.,1985.0,1,13.191004,4.888167,52.420488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167971,240,130.0,120,200,208,128,9,246,0,-1,...,1.0,0.0,0.0,0.0,e. 15 tot 18 m.,1953.0,1,18.854421,4.824533,52.376163
167977,180,60.0,60,100,166,155,0,0,4,1,...,1.0,0.0,0.0,0.0,b. 6 tot 9 m.,0.0,1,19.252113,4.983032,52.310373
168077,300,110.0,100,170,142,120,0,0,5,3,...,1.0,0.0,0.0,0.0,b. 6 tot 9 m.,0.0,1,5.597120,4.917106,52.368343
168846,110,30.0,30,50,247,163,7,23,0,0,...,0.0,0.0,0.0,0.0,e. 15 tot 18 m.,1950.0,1,8.968628,4.896493,52.370812


#### Tree Data <-> Non-Incidents

In [ ]:
# Making a copy of the non_incident_data to avoid modifying the original DataFrame
non_incident_data_mod = non_incident_data.copy()

# Convert 'Date' to datetime and extract the year
non_incident_data_mod["Year"] = pd.to_datetime(non_incident_data_mod["Date"]).dt.year

# Generate a list to store random tree indices
random_tree_indices = []

# Iterate over each row in non-incident data to find a valid random tree index
for year in tqdm(non_incident_data_mod["Year"]):
    valid_trees = tree_data[tree_data["jaarVanAanleg"] <= year]
    if not valid_trees.empty:
        random_index = np.random.choice(valid_trees.index)
        random_tree_indices.append(random_index)
    else:
        random_tree_indices.append(np.nan)
# Use the random indices to fetch the corresponding tree rows
random_trees = tree_data.loc[random_tree_indices]

# Reset the index of non_incident_data_mod and random_trees for concatenation
non_incident_data_mod.reset_index(drop=True, inplace=True)
random_trees.reset_index(drop=True, inplace=True)

# Concatenate the non-incident data with the randomly selected tree data
augmented_non_incidents = pd.concat([non_incident_data_mod, random_trees], axis=1)
augmented_non_incidents.head()

In [111]:
# Making a copy of the non_incident_data to avoid modifying the original DataFrame
non_incident_data_mod = non_incident_data.copy()

# Convert 'Date' to datetime and extract the year
non_incident_data_mod["Year"] = pd.to_datetime(non_incident_data_mod["Date"]).dt.year

# Generate random tree indices efficiently
valid_trees_indices = (
    tree_data["jaarVanAanleg"].searchsorted(non_incident_data_mod["Year"], side="right")
    - 1
)
valid_trees_indices = np.clip(valid_trees_indices, 0, len(tree_data) - 1)
random_tree_indices = np.random.choice(
    valid_trees_indices, size=len(non_incident_data_mod)
)

# Use the random indices to fetch the corresponding tree rows
random_trees = tree_data.loc[random_tree_indices].reset_index(drop=True)

# Reset the index of non_incident_data_mod for concatenation
non_incident_data_mod.reset_index(drop=True, inplace=True)

# Concatenate the non-incident data with the randomly selected tree data
augmented_non_incidents = pd.concat([non_incident_data_mod, random_trees], axis=1)
augmented_non_incidents.head()

,Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,...,jaarVanAanleg,soortnaam,stamdiameterklasse,standplaatsGedetailleerd,typeObject,typeSoortnaam,soortnaamKort,soortnaamTop,LON,LAT
0,240,2005-01-01,1,260,40.0,30,60,68,NaN,57,...,0.0,Onbekend,None,None,None,Bomen,Onbekend,Onbekend,4.948994,52.363003
1,240,2005-01-01,2,230,30.0,30,60,65,NaN,52,...,0.0,Onbekend,None,None,None,Bomen,Onbekend,Onbekend,4.948994,52.363003
2,240,2005-01-01,3,230,40.0,30,50,43,NaN,34,...,0.0,Onbekend,None,None,None,Bomen,Onbekend,Onbekend,4.948994,52.363003
3,240,2005-01-01,4,220,40.0,40,50,38,NaN,32,...,2005.0,Ulmus minor,None,Tegels,Boom niet vrij uitgroeiend,Bomen,Ulmus,Iep (Ulmus),4.886882,52.415793
4,240,2005-01-01,5,230,40.0,40,50,38,NaN,34,...,2018.0,None,None,None,Boom vrij uitgroeiend,Bomen,None,None,4.832758,52.382500
